In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import math
import os
import glob
import keras

from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [25]:
# Load and build raw data structures
path = glob.glob('./testImages/class_a/*.jpg')
equImages = []
equLabels = []
for img in path:
    i = cv2.imread(img)
    iResized = cv2.resize(i, (256,256))
    equImages.append(iResized)
    equLabels.append(0)
    
equX = np.asarray(equImages).reshape((len(equImages), 256, 256, 3)) 
equY = np.asarray(equLabels)

In [27]:
print(equX.shape)
print(equY.shape)

(38, 256, 256, 3)
(38,)


In [19]:
# Data Augmentation on the fly


In [ ]:
# Build the model

In [ ]:
# Train the model 